In [2]:
import torch
from transformers import pipeline

/Users/victoria/Documents/nlp_course_hw/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [96]:
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.float16, device_map="auto")

In [97]:
system_content =  ("For a given sentence, create a set of extended sentences with approximately the same meaning. "
                "Your answer should contains only numbered list of these extended sentences. "
                "For example, "
                "user message: After your workout, remember to focus on maintaining a good water balance. "
                "Your answer: 1.Remember to drink enough water to restore and maintain your body's hydration after your cardio training. "
                "2.Please don't forget to maintain water balance after workout.")
print(system_content)

For a given sentence, create a set of extended sentences with approximately the same meaning. Your answer should contains only numbered list of these extended sentences. For example, user message: After your workout, remember to focus on maintaining a good water balance. Your answer: 1.Remember to drink enough water to restore and maintain your body's hydration after your cardio training. 2.Please don't forget to maintain water balance after workout.


In [98]:
user_content = "A good night's sleep helps improve your mood, increase concentration and improve memory."

In [99]:
messages = [
    {
        "role": "system",
        "content":  system_content,
    },
    {"role": "user", "content": user_content},
]

In [100]:
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

outputs = pipe(prompt, max_new_tokens=200, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

In [101]:
generated_text = outputs[0]["generated_text"]

In [102]:
print(generated_text)

<|system|>
For a given sentence, create a set of extended sentences with approximately the same meaning. Your answer should contains only numbered list of these extended sentences. For example, user message: After your workout, remember to focus on maintaining a good water balance. Your answer: 1.Remember to drink enough water to restore and maintain your body's hydration after your cardio training. 2.Please don't forget to maintain water balance after workout.</s>
<|user|>
A good night's sleep helps improve your mood, increase concentration and improve memory.</s>
<|assistant|>
Sentence: A good night's sleep helps improve your mood, increases concentration, and improves memory.

1. Remember to get enough restful sleep to help your brain function at its best.
2. Make sleep a priority in your routine.
3. Keep your bedroom cool and dark to promote better sleep.
4. Avoid caffeine and alcohol before bedtime.
5. Establish a consistent bedtime routine to help your body and mind relax.
6. Use

In [106]:
text = generated_text.replace("<|system|>", "")
text = text.replace("<|user|>", "")
text = text.replace("<|assistant|>", "")
print(text)


For a given sentence, create a set of extended sentences with approximately the same meaning. Your answer should contains only numbered list of these extended sentences. For example, user message: After your workout, remember to focus on maintaining a good water balance. Your answer: 1.Remember to drink enough water to restore and maintain your body's hydration after your cardio training. 2.Please don't forget to maintain water balance after workout.</s>

A good night's sleep helps improve your mood, increase concentration and improve memory.</s>

Sentence: A good night's sleep helps improve your mood, increases concentration, and improves memory.

1. Remember to get enough restful sleep to help your brain function at its best.
2. Make sleep a priority in your routine.
3. Keep your bedroom cool and dark to promote better sleep.
4. Avoid caffeine and alcohol before bedtime.
5. Establish a consistent bedtime routine to help your body and mind relax.
6. Use a sleep-inducing app or light 

In [121]:
remove_system_prompt = text.replace(system_content,"")
print(remove_system_prompt)


</s>

A good night's sleep helps improve your mood, increase concentration and improve memory.</s>

Sentence: A good night's sleep helps improve your mood, increases concentration, and improves memory.

1. Remember to get enough restful sleep to help your brain function at its best.
2. Make sleep a priority in your routine.
3. Keep your bedroom cool and dark to promote better sleep.
4. Avoid caffeine and alcohol before bedtime.
5. Establish a consistent bedtime routine to help your body and mind relax.
6. Use a sleep-inducing app or light therapy device to promote better sleep.
7. Avoid daytime naps that can disrupt your sleep schedule.
8. Create a relaxing bedtime routine, such as reading or listening to music, to help you wind down.
9. Don't forget to establish a consistent sleep schedule.
10. Limit alcohol and caffeine intake before bedtime


In [122]:
cleaned = remove_system_prompt.replace("\n","")
cleaned = cleaned.replace("</s>","")
print(cleaned)

A good night's sleep helps improve your mood, increase concentration and improve memory.Sentence: A good night's sleep helps improve your mood, increases concentration, and improves memory.1. Remember to get enough restful sleep to help your brain function at its best.2. Make sleep a priority in your routine.3. Keep your bedroom cool and dark to promote better sleep.4. Avoid caffeine and alcohol before bedtime.5. Establish a consistent bedtime routine to help your body and mind relax.6. Use a sleep-inducing app or light therapy device to promote better sleep.7. Avoid daytime naps that can disrupt your sleep schedule.8. Create a relaxing bedtime routine, such as reading or listening to music, to help you wind down.9. Don't forget to establish a consistent sleep schedule.10. Limit alcohol and caffeine intake before bedtime


In [123]:
import re
rm_list = re.sub(r'[0-9]+. ', '', cleaned)
print(rm_list)

A good night's sleep helps improve your mood, increase concentration and improve memory.Sentence: A good night's sleep helps improve your mood, increases concentration, and improves memory.Remember to get enough restful sleep to help your brain function at its best.Make sleep a priority in your routine.Keep your bedroom cool and dark to promote better sleep.Avoid caffeine and alcohol before bedtime.Establish a consistent bedtime routine to help your body and mind relax.Use a sleep-inducing app or light therapy device to promote better sleep.Avoid daytime naps that can disrupt your sleep schedule.Create a relaxing bedtime routine, such as reading or listening to music, to help you wind down.Don't forget to establish a consistent sleep schedule.Limit alcohol and caffeine intake before bedtime


In [127]:
sentences = rm_list.split(".")
meanful = []
for sentence in sentences:
    if len(sentence.split()) > 2:
        meanful.append(sentence)

In [128]:
meanful

["A good night's sleep helps improve your mood, increase concentration and improve memory",
 "Sentence: A good night's sleep helps improve your mood, increases concentration, and improves memory",
 'Remember to get enough restful sleep to help your brain function at its best',
 'Make sleep a priority in your routine',
 'Keep your bedroom cool and dark to promote better sleep',
 'Avoid caffeine and alcohol before bedtime',
 'Establish a consistent bedtime routine to help your body and mind relax',
 'Use a sleep-inducing app or light therapy device to promote better sleep',
 'Avoid daytime naps that can disrupt your sleep schedule',
 'Create a relaxing bedtime routine, such as reading or listening to music, to help you wind down',
 "Don't forget to establish a consistent sleep schedule",
 'Limit alcohol and caffeine intake before bedtime']

In [129]:
best_cases = meanful[2:]
best_cases

['Remember to get enough restful sleep to help your brain function at its best',
 'Make sleep a priority in your routine',
 'Keep your bedroom cool and dark to promote better sleep',
 'Avoid caffeine and alcohol before bedtime',
 'Establish a consistent bedtime routine to help your body and mind relax',
 'Use a sleep-inducing app or light therapy device to promote better sleep',
 'Avoid daytime naps that can disrupt your sleep schedule',
 'Create a relaxing bedtime routine, such as reading or listening to music, to help you wind down',
 "Don't forget to establish a consistent sleep schedule",
 'Limit alcohol and caffeine intake before bedtime']

In [135]:
mask_pipe = pipeline("fill-mask", model="distilroberta-base")


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [143]:
from random import shuffle, choice

In [145]:
def mask_sentence(sentence: str) -> str:
    mask = '<mask>'
    words = sentence.split()
    words = [word for word in words if len(word) > 3]
    shuffle(words)
    masked_candidate = choice(words)
    masked = sentence.replace(masked_candidate, mask, 1)
    return masked

In [155]:
potential = []
for sentence in best_cases:
    masked = mask_sentence(sentence)
    output = mask_pipe(masked)
    output = output[:2]
    sequences = [response["sequence"] for response in output]
    potential.append(sequences)
    
    

In [156]:
potential

[['Remember to get enough restful sleep to help your brain function at its best',
  'Remember to get enough restful meals to help your brain function at its best'],
 ['Make sleep a priority in your routine',
  'Make sleep a factor in your routine'],
 ['Keep your bedroom cool and dark to ensure better sleep',
  'Keep your bedroom cool and dark to get better sleep'],
 ['Avoid caffeine and alcohol before bedtime',
  'Limit caffeine and alcohol before bedtime'],
 ['Develop a consistent bedtime routine to help your body and mind relax',
  'Create a consistent bedtime routine to help your body and mind relax'],
 ['Use a sleep-inducing app or sleep therapy device to promote better sleep',
  'Use a sleep-inducing app or relaxation therapy device to promote better sleep'],
 ['Avoid daytime naps that can disrupt your sleep schedule',
  'Avoid daytime naps which can disrupt your sleep schedule'],
 ['Create a relaxing bedtime routine, such as reading or listening to music, to help you wind down',
